In [3]:
#Ideas
#types of steps, length of steps, types of ingredients length of ingredients


In [4]:
import pandas as pd
import numpy as np

CSV_LEN = 6

def read_data(file):
    df = pd.read_csv(file) 
    X = []
    y = []
    if df.shape[1] == CSV_LEN:
        y = df.loc[:, 'duration_label']
        X = df.iloc[:,:CSV_LEN-1]
    else:
        X = df.iloc[:,:CSV_LEN-1]
    return X, y

In [5]:
X, y = read_data('recipe_train.csv')


In [203]:
from sklearn.svm import SVC

class model1:
    def __init__(self):
        self.model = SVC(gamma=2, C=1)
        
    def fit(self,X,y):
        mod_X = X.loc[:,['n_steps','n_ingredients']]
        self.model.fit(mod_X,y)
        
    def predict(self,x):
        mod_x = x[['n_steps','n_ingredients']]
        return self.model.predict(mod_x.values.reshape(1, -1))[0]
        
    def score(self,X,y):
        mod_X = X.loc[:,['n_steps','n_ingredients']]
        return self.model.score(mod_X,y)
        
    

In [237]:
import time
toc = time.perf_counter()
model = model1()
model.fit(X,y)
model.score(X,y)
print(0.64405)
toc = time.perf_counter()
print(toc-tic)

0.64405
15618.083697454998


In [164]:
from sklearn.tree import DecisionTreeClassifier

class model2:
    def __init__(self):
        self.model =  DecisionTreeClassifier(random_state=0)
        
    def fit(self,X,y):
        mod_X = X.loc[:,['n_steps','n_ingredients']]
        self.model.fit(mod_X,y)
        
    def predict(self,x):
        mod_x = x[['n_steps','n_ingredients']]
        return self.model.predict(mod_x.values.reshape(1, -1))[0]
        
    def score(self,X,y):
        mod_X = X.loc[:,['n_steps','n_ingredients']]
        return self.model.score(mod_X,y)

In [238]:
tic = time.perf_counter()
model = model2() 
model.fit(X,y)
model.score(X,y)
toc = time.perf_counter()
print(toc-tic)
print(0.644125)

0.04944610700113117
0.644125


In [163]:
train_corpus_name = X.loc[0:10,['name', 'steps','ingredients']]
corpus_name = list(tokenize_corpus(train_corpus_name))

In [130]:
#doc2vec provided code
import gensim
import ast
# function to preprocess and tokenize text
def tokenize_corpus(txt, tokens_only=False):
    if txt.shape[0] == txt.size:
        line = txt
        name = line[0]
        steps=""
        for i in ast.literal_eval(line[1]):
            steps+= " "+i
        ingeredients=""
        for i in ast.literal_eval(line[2]):
            ingeredients+=" "+i
        row = name + " " + steps + " " + ingeredients
        tokens = gensim.utils.simple_preprocess(row)
        if tokens_only:
            yield tokens
        else:
            yield gensim.models.doc2vec.TaggedDocument(tokens, [line[0]])
        return
    for line in txt.iterrows():
        name = line[1][0]
        steps=""
        for i in ast.literal_eval(line[1][1]):
            steps+= " "+i
        ingeredients=""
        for i in ast.literal_eval(line[1][2]):
            ingeredients+=" "+i
        row = name + " " + steps + " " + ingeredients
        tokens = gensim.utils.simple_preprocess(row)
        if tokens_only:
            yield tokens
        else:
            yield gensim.models.doc2vec.TaggedDocument(tokens, [line[0]])

In [131]:
#doc2vec provided code
import gensim
from sklearn.tree import DecisionTreeClassifier
    
class model3:
    def __init__(self):
        self.vec_size = 75
        self.dtmodel =  DecisionTreeClassifier(random_state=0)
        self.genmodel = gensim.models.doc2vec.Doc2Vec(vector_size=self.vec_size, min_count=1, epochs=40)
        
    def fit(self,X,y):
        train_corpus_name = X.loc[:,['name', 'steps','ingredients']]
        test_name = X.loc[:,['name','steps','ingredients']]

        # tokenize a training corpus
        corpus_name = list(tokenize_corpus(train_corpus_name))
        # train doc2vec on the training corpus
        self.genmodel.build_vocab(corpus_name)
        self.genmodel.train(corpus_name, total_examples=self.genmodel.corpus_count, epochs=self.genmodel.epochs)

        # tokenize new documents
        doc = list(tokenize_corpus(test_name, tokens_only=True))
        # generate embeddings for the new documents
        x_test_name = np.zeros((len(doc),self.vec_size))
        for i in range(len(doc)):
            x_test_name[i,:] = self.genmodel.infer_vector(doc[i])
        self.dtmodel.fit(x_test_name,y)
        
    def predict(self,x):
        doc = list(tokenize_corpus(x.loc[['name', 'steps','ingredients']], tokens_only=True))
        x_test_name = np.zeros((1,self.vec_size))
        x_test_name[0,:] = self.genmodel.infer_vector(doc[0])
        return self.dtmodel.predict(x_test_name)[0]
        
    def score(self,X,y):
        doc = list(tokenize_corpus(X.loc[:,['name', 'steps','ingredients']], tokens_only=True))
        x_test_name = np.zeros((len(doc),self.vec_size))
        for i in range(len(doc)):
            x_test_name[i,:] = self.genmodel.infer_vector(doc[i])
        return self.dtmodel.score(x_test_name,y)
    


In [186]:
import csv
def write_output(model):
    test = "recipe_test.csv"
    X, y = read_data(test)
    f = open('output.csv', 'w')
    writer = csv.writer(f)
    writer.writerow(['id','duration_label'])
    #for i, row in enumerate(X.iterrows()):
    for i in range(10000):
        x = X.loc[i,:]
        writer.writerow([i+1,model.predict(x)])
    f.close()
    


In [14]:
tokenize_corpus(X.loc[0]['name'], tokens_only=True)

<generator object tokenize_corpus at 0x119d56480>

In [128]:
import time

tic = time.perf_counter()
model =  model3()
model.fit(X,y)
#model.score(X,y)
toc = time.perf_counter()
print(toc-tic)

540.1526337899995


In [233]:
def combine_features(txt):
    combined =[]
    if txt.shape[0] == txt.size:
        line = txt
        name = line[0]
        steps=""
        for i in ast.literal_eval(line[1]):
            steps+= " "+i
        ingeredients=""
        for i in ast.literal_eval(line[2]):
            ingeredients+=" "+i
        row = name + " " + steps + " " + ingeredients
        combined.append(row)
        return combined
    for line in txt.iterrows():
        name = line[1][0]
        steps=""
        for i in ast.literal_eval(line[1][1]):
            steps+= " "+i
        ingeredients=""
        for i in ast.literal_eval(line[1][2]):
            ingeredients+=" "+i
        row = name + " " + steps + " " + ingeredients
        combined.append(row)
    return combined


In [239]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import SGDClassifier

class model4:
    def __init__(self):
        self.model = SGDClassifier(max_iter = 1000, alpha = 0.001)
        self.vocab = CountVectorizer(stop_words='english')
        
    def fit(self,X,y):
        train_corpus = X.loc[:,['name', 'steps','ingredients']]

        # fit the count vec
        self.vocab.fit(combine_features(train_corpus))
        
        #vectorise each row
        X_mod = self.vocab.transform(combine_features(train_corpus))
        # fit
        self.model.fit(X_mod,y)
        
    def predict(self,x):
        row  = x.loc[['name', 'steps','ingredients']]
        mod_x = self.vocab.transform(combine_features(row))
        return self.model.predict(mod_x)[0]
        
    def score(self,X,y):
        data = X.loc[:,['name', 'steps','ingredients']]
        X_mod = self.vocab.transform(combine_features(data))
        return self.model.score(X_mod,y)
    

In [240]:
toc = time.perf_counter()
model = model4() 
model.fit(X,y)
print(model.score(X,y))
toc = time.perf_counter()
print(toc-tic)
print(0.843175)
write_output(model)

1.0
1133.6593502450014
0.843175


In [195]:

from sklearn.model_selection import train_test_split

def random_cross_val_score(model, X, y, kfold, itter):
    score = 0
    for i in range(itter):
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=1/kfold, random_state=42)
        model.fit(X_train,y_train)
        score += model.score(X_test,y_test)
        print(score/(i+1))
    return score/itter

In [196]:
random_cross_val_score(model, X, y, 10, 10)

0.8015
0.801625
0.8000833333333334
0.8008125
0.80095
0.80125
0.8013928571428571
0.80140625
0.8009999999999999
0.8006499999999999


0.8006499999999999

In [197]:
random_cross_val_score(model1(), X, y, 10, 10)

0.63725
0.63725
0.63725
0.63725
0.63725
0.63725
0.63725
0.63725
0.63725
0.63725


0.63725

In [198]:
random_cross_val_score(model2(), X, y, 10, 10)

0.6305
0.6305
0.6305
0.6305
0.6305
0.6305
0.6305
0.6305
0.6305
0.6304999999999998


0.6304999999999998

In [241]:
write_output(model)